In [2]:
from linkml.generators.jsonldcontextgen import ContextGenerator

# aren't the dumpers in a differnt path in the non-CCDH linkml now?
from linkml_runtime.dumpers import json_dumper
from linkml_runtime.dumpers import yaml_dumper

# was this problematic in a GH action? 
from rdflib import Graph
# /Users/MAM/.local/share/virtualenvs/example-data-WqQDUTxX/lib/python3.9/site-packages/rdflib_jsonld/__init__.py:9: DeprecationWarning: The rdflib-jsonld package has been integrated into rdflib as of rdflib==6.0.0.  Please remove rdflib-jsonld from your project's dependencies.
#   warnings.warn(

import json
import pandas
import sys

# change this!
# required at Loading the Python classes for the CCDH model
# from ccdh import ccdhmodel as ccdh
import crdch_model as ccdh

ModuleNotFoundError: No module named 'crdch_model'

# GDC to CCDH Conversion

This notebook demonstrates one method for converting GDC data into CCDH (CRDC-H) instance data: by reading node data as JSON and writing it out in the LinkML model. The LinkML can be used to [generate](https://github.com/linkml/linkml#python-dataclasses) [Python Data Classes](https://docs.python.org/3/library/dataclasses.html), which can then be exported in JSON-LD, a JSON-based format used to represent RDF data.

## Why Python Data Classes?

Python Data Classes provide several useful features that we will demonstrate below:

1. **Python Data Classes are generated automatically.** Rather than requiring additional effort to maintain a Python library for accessing the CCDH model, the [LinkML toolset](https://linkml.github.io/) can generate the Python Data Classes directly from the CCDH model, ensuring that users can always access the most recent version of the CCDH model programmatically. This also allows us to maintain Python Data Classes for accessing previous versions of the CCDH model, which we plan to use to implement [data migration between CCDH model versions](https://cancerdhc.github.io/ccdhmodel/latest/data-migration/)
2. **Python Data Classes provide validation on creation.** As we will demonstrate below, creating a Python Data Class requires that all required attributes are filled in, and all fields are filled in the format or enumeration expected.
3. **Easy to use in Python IDEs.** Since the generated Python Data Classes includes model documentation in Python, users using Python IDEs can see available options and documentation while writing their code.

## Setup

We start by installing the [LinkML](https://pypi.org/project/linkml/) and [pandas](https://pypi.org/project/pandas/) packages. This is included in the pipenv file included in this source repository: if you used `pipenv run jupyter notebook` to start this Notebook, you should be set up already. If not, you may need to uncomment these following lines to install these packages.

(Note: if you are running this on macOS 11 "Big Sur", you might need to set `SYSTEM_VERSION_COMPAT=1` in your Terminal environment before running `pipenv install`).

In [ ]:
import sys

# Install LinkML.
# We use our own fork of LinkML, but all changes made to this repository will eventually be sent
# upstream to the main LinkML release.
#!{sys.executable} -m pip install linkml

# Install pandas.
#!{sys.executable} -m pip install pandas

# Install rdflib.
#!{sys.executable} -m pip install rdflib

# Install JSON Schema.
#!{sys.executable} -m pip install jsonschema

## Loading GDC data as an example

In this demonstration, we will use a dataset of 560 cases relating to head and neck cancers previously downloaded from the public GDC API as [documented elsewhere in this repository](https://github.com/cancerDHC/example-data/blob/main/head-and-mouth/Head%20and%20Mouth%20Cancer%20Datasets.ipynb).

In [ ]:
import json
import pandas

with open('head-and-mouth/gdc-head-and-mouth.json') as file:
    gdc_head_and_mouth = json.load(file)
    
pandas.DataFrame(gdc_head_and_mouth)

## Loading the Python classes for the CCDH model

The Python DataClasses for the CCDH model as available at https://github.com/cancerDHC/ccdhmodel/. The Python DataClasses cannot be directly loaded from this GitHub repository yet, but we [plan to implement this functionality soon](https://github.com/cancerDHC/ccdhmodel/issues/40). For now, we have copied the file into this repository so we can import them here.

Note that the Python Data Classes includes documentation on entities and enumerations.

In [ ]:
# see very first import block in notebook for replacement
# from ccdh import ccdhmodel as ccdh

# Documentation for an entity.
print(f"Documentation for Specimen: {ccdh.Specimen.__doc__}")

# Documentation for an enumeration.
print(f"Documentation for Specimen.specimen_type: {ccdh.EnumCCDHSpecimenSpecimenType.__doc__}")

# List of permissible values for Specimen.specimen_type
print("Permissible values in enumeration Specimen.specimen_type:")
pvalues = [pv for key, pv in ccdh.EnumCCDHSpecimenSpecimenType.__dict__.items() if isinstance(pv, ccdh.PermissibleValue)]
for pv in pvalues:
    print(f' - Value "{pv.text}": {pv.description}')

## Transforming GDC cases into CCDH Research Subject

The primary transformation we will demonstrate here is transforming a [GDC case](https://docs.gdc.cancer.gov/Data_Dictionary/viewer/#?view=table-definition-view&id=case) into a [CCDH Research Subject](https://cancerdhc.github.io/ccdhmodel/latest/ResearchSubject/). To do this, we need to translate three additional components as well:
* Each GDC case includes a diagnosis, which we need to transform into a [CCDH Diagnosis](https://cancerdhc.github.io/ccdhmodel/latest/Diagnosis/).
* Each GDC diagnosis includes a description of the cancer stage (see properties named `ajcc_*` in [the GDC documentation](https://docs.gdc.cancer.gov/Data_Dictionary/viewer/#?view=table-definition-view&id=diagnosis)). We will translate this into a [CCDH Cancer Stage Observation Set](https://cancerdhc.github.io/ccdhmodel/latest/CancerStageObservationSet/).
* Each GDC case contains a hierarchy of samples, portions, analytes, aliquots and slides. For the purposes of this demonstration, we will focus on transforming only the top-level specimens into [CCDH Specimens](https://cancerdhc.github.io/ccdhmodel/latest/Specimen/), but the same method can be used to transform other parts of the hierarchy. We plan to [include that transformation](https://github.com/cancerDHC/example-data/issues/6) in this tutorial eventually. Note that in our model, specimens are associated with diagnoses rather than directly with Research Subjects.

The CCDH Python Data Classes help in writing these transformation methods by applying validation on the data and ensuring that constraints (such as the required fields) are met. We begin by defining a transformation for creating a [CCDH BodySite](https://cancerdhc.github.io/ccdhmodel/latest/BodySite/), which we also use to demonstrate the validation features available on CCDH Python Data Classes.

In [ ]:
def create_body_site(site_name):
    """ Create a CCDH BodySite based on the name of a site in the human body."""
    
    # Accept 'None'.
    if site_name is None:
        return None
    
    # Some body sites are not currently included in the CCDH model. We will need to translate these sites
    # into values that *are* included in the CCDH model.
    site_mappings = {
        'Larynx, NOS': ccdh.EnumCCDHBodySiteSite.Larynx
    }
    
    # Map values if needed. Otherwise, pass them through unmapped.
    if site_name in site_mappings:
        return ccdh.BodySite(site=(site_mappings[site_name]))
    
    return ccdh.BodySite(site=site_name)

# Try to create a body site for a site name not currently included in the CCDH model.
try:
    create_body_site('Laryn') # Note misspelling.
except ValueError as v:
    print(f'Could not create BodySite: {v}')

# Using a valid name generates no errors.
create_body_site('Larynx')    

# Using a mapped name generates no errors, as it is mapped to a valid name.
create_body_site('Larynx, NOS')

We need a more sophisticated transformation method for transforming the GDC cancer stage information into [CCDH Cancer Stage Observation Set](https://cancerdhc.github.io/ccdhmodel/latest/CancerStageObservationSet/). Each observation set is made up of a number of [CCDH Cancer Stage Observations](https://cancerdhc.github.io/ccdhmodel/latest/CancerStageObservation/), each of which represents a different type of observation.

In [ ]:
def create_stage_observation(type, value):
    """ Create a CCDHCancerStageObservation from a type of observation and a codeable concept."""
    # As with the body site example above, we need to map GDC values into the values
    # allowed under the CCDH model.
    stage_mappings = {
        'not reported': 'Not Reported',
        'unknown': 'Unknown',
        'stage i': 'Stage I',
        'stage ii': 'Stage II',
        'stage iii': 'Stage III',
        'stage iva': 'Stage IVA',
        'stage ivb': 'Stage IVB',
        'stage ivc': 'Stage IVC',
    }
    
    if value in stage_mappings:
        return ccdh.CancerStageObservation(
            observation_type=type,
            valueCodeableConcept=stage_mappings[value]
        )
    
    return ccdh.CancerStageObservation(
        observation_type=type,
        valueCodeableConcept=value
    )

def create_stage_from_gdc(diagnosis):
    cancer_stage_method_type = None
    if diagnosis.get('ajcc_staging_system_edition') == '7th':
        cancer_stage_method_type = 'AJCC staging system 7th edition'

    # Create an observation set 
    obs = ccdh.CancerStageObservationSet(
        method_type=cancer_stage_method_type
    )
    
    # Add observations for every type of observation in the GDC diagnosis.
    if diagnosis.get('tumor_stage') is not None:
        obs.observations.append(create_stage_observation('Overall', diagnosis.get('tumor_stage')))
        
    if diagnosis.get('ajcc_clinical_stage') is not None:
        obs.observations.append(create_stage_observation('Clinical Overall', diagnosis.get('ajcc_clinical_stage')))
        
    if diagnosis.get('ajcc_clinical_t') is not None:
        obs.observations.append(create_stage_observation('Clinical Tumor (T)', diagnosis.get('ajcc_clinical_t')))
        
    if diagnosis.get('ajcc_clinical_n') is not None:
        obs.observations.append(create_stage_observation('Clinical Node (N)', diagnosis.get('ajcc_clinical_n')))
        
    if diagnosis.get('ajcc_clinical_m') is not None:
        obs.observations.append(create_stage_observation('Clinical Metastasis (M)', diagnosis.get('ajcc_clinical_m')))
    
    if diagnosis.get('ajcc_pathologic_stage') is not None:
        obs.observations.append(create_stage_observation('Pathological Overall', diagnosis.get('ajcc_pathologic_stage')))
        
    if diagnosis.get('ajcc_pathologic_t') is not None:
        obs.observations.append(create_stage_observation('Pathological Tumor (T)', diagnosis.get('ajcc_pathologic_t')))
        
    if diagnosis.get('ajcc_pathologic_n') is not None:
        obs.observations.append(create_stage_observation('Pathological Node (N)', diagnosis.get('ajcc_pathologic_n')))
        
    if diagnosis.get('ajcc_pathologic_m') is not None:
        obs.observations.append(create_stage_observation('Pathological Metastasis (M)', diagnosis.get('ajcc_pathologic_m')))
    
    return obs

# Test transform with the diagnosis from the first loaded case.
# Note that the resulting CancerStageObservationSet contains descriptions for the concepts included in it.
# example_observation_set = create_stage_from_gdc(gdc_head_and_mouth[131]['diagnoses'][0], ccdh.Subject(id='1234'))
example_observation_set = create_stage_from_gdc(gdc_head_and_mouth[131]['diagnoses'][0])
example_observation_set

Reading Python Data Classes in its default text output can be difficult! However, we can use LinkML's [YAML](https://en.wikipedia.org/wiki/YAML) dumper to display this Cancer Stage Observation Set as a YAML string. YAML objects are a good way to export LinkML data, and include detailed descriptions of all the enumerations referenced from this object. We currently include basic descriptions for the permissible values (see e.g. "N1 Stage Finding" below), but we will include more detailed descriptions in the future.

In [ ]:
from linkml_runtime.dumpers import yaml_dumper

print(yaml_dumper.dumps(example_observation_set))

Diagnoses can contain samples, which we transform into [CCDH Samples](https://cancerdhc.github.io/ccdhmodel/latest/Specimen/).

In [ ]:
def transform_sample_to_specimen(sample):
    """
    A method for transforming a GDC Sample into CCDH Specimen.
    """
    specimen = ccdh.Specimen(id = sample.get('sample_id'))
    specimen.source_material_type = sample.get('sample_type')
    specimen.general_tissue_morphology = sample.get('tissue_type')
    specimen.specific_tissue_morphology = sample.get('tumor_code')
    specimen.tumor_status_at_collection = sample.get('tumor_descriptor')
    specimen.creation_activity = ccdh.SpecimenCreationActivity(
        date_ended=ccdh.TimePoint(
            dateTime=sample.get('created_datetime')
        )
    )
    return specimen

# Let's try creating a test specimen.
test_specimen = transform_sample_to_specimen(gdc_head_and_mouth[2]['samples'][0])
test_specimen

We can now transform an entire diagnosis into a [CCDH Diagnosis](https://cancerdhc.github.io/ccdhmodel/latest/Diagnosis/).

In [ ]:
def transform_diagnosis(diagnosis, case):
    ccdh_diagnosis = ccdh.Diagnosis(
        id=diagnosis.get('diagnosis_id'),
        condition=diagnosis.get('primary_diagnosis'),
        morphology=diagnosis.get('morphology'),
        grade=diagnosis.get('grade'),
        stage=create_stage_from_gdc(diagnosis),
        year_at_diagnosis=diagnosis.get('year_of_diagnosis'),
        related_specimen=[
            transform_sample_to_specimen(
                sample
            ) for sample in case.get('samples')
        ]
    )
    ccdh_diagnosis.identifier = [
        ccdh.Identifier(
            system='GDC-submitter-id',
            value=diagnosis.get('submitter_id')
        )
    ]
    
    if 'primary_site' in case and case['primary_site'] != '':
        body_site = create_body_site(case['primary_site'])
        if body_site is not None:
            ccdh_diagnosis.metastatic_site.append(body_site)

    return ccdh_diagnosis

example_diagnosis = transform_diagnosis(gdc_head_and_mouth[131]['diagnoses'][0], gdc_head_and_mouth[131])
print(yaml_dumper.dumps(example_diagnosis))

## Exporting Python Data Classes as JSON-LD

Python Data Classes can be exported as [JSON-LD](https://en.wikipedia.org/wiki/JSON-LD), allowing CCDH instance data to be shared in a [JSON](https://en.wikipedia.org/wiki/JSON)-based [RDF](https://en.wikipedia.org/wiki/Resource_Description_Framework) format. RDF formats are particularly useful in sharing data, since they allow us to share [Linked Data](https://en.wikipedia.org/wiki/Linked_data) that can be understood by other consumers.

In [ ]:
from linkml.generators.jsonldcontextgen import ContextGenerator
from linkml_runtime.dumpers import json_dumper

jsonldContext = ContextGenerator('ccdh/ccdhmodel.yaml').serialize()
jsonldContextAsDict = json.loads(jsonldContext)

# Display the example diagnosis we constructed in a previous step.
print(json_dumper.dumps(example_diagnosis, jsonldContext))

We can also transform all the diagnoses in this file and store them in a file as JSON-LD.

In [ ]:
diagnoses = []
for case in gdc_head_and_mouth:
    for diagnosis in case['diagnoses']:
        diagnoses.append(transform_diagnosis(diagnosis, case))

jsonld = json_dumper.dumps({
    '@graph': diagnoses,
    '@context': jsonldContextAsDict
})
with open('head-and-mouth/diagnoses.jsonld', 'w') as file:
    file.write(jsonld)

## Converting JSON-LD to Turtle

While JSON-LD is a full dialect of RDF, people are more familiar looking at RDF in a format like [Turtle](https://en.wikipedia.org/wiki/Turtle_(syntax)). We can convert the generated JSON-LD output into Turtle by using the [rdflib](https://rdflib.readthedocs.io/en/stable/) package.

Note that this section is intended to be illustrative -- these are *not* finalized IRIs for properties and entities. We will choose IRIs and develop a canonical RDF representation in future phases of development.

In [ ]:
# We can read this JSON-LD in Turtle.
from rdflib import Graph

g = Graph()
g.parse(data=jsonld, format="json-ld")
rdfAsTurtle = g.serialize(format="turtle").decode()
print(''.join(rdfAsTurtle[0:1000]))
with open('head-and-mouth/diagnoses.ttl', 'w') as file:
    file.write(rdfAsTurtle)